# Convolutional Neural Network

#### Library import 

In [ ]:
import tensorflow as tf
import numpy as np

#### Load data 

In [ ]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

#### Variables declarations

In [ ]:
n_classes = 10
batch_size = 128

# height * width
X = tf.placeholder(tf.float32, [None, 784])
y = tf.placeholder(tf.float32)

keep_rate = 0.8
keep_prob = tf.placeholder(tf.float32)

#### Fuction define for creating neural network layers

In [ ]:
def conv2d(X, W):
    return tf.nn.conv2d(X, W, strides=[1,1,1,1], padding='SAME')

#### Define maxpooling function 

In [ ]:
def maxpool2d(X):
    return tf.nn.max_pool(X, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')

#### Neural Network model build 

In [ ]:
def convolutional_neural_networks(X):
    weights = {'w_conv1': tf.Variable(tf.random_normal([5, 5, 1, 32])),
               'w_conv2': tf.Variable(tf.random_normal([5, 5, 32, 64])),
               'w_fc': tf.Variable(tf.random_normal([7*7*64, 1024])),
               'out': tf.Variable(tf.random_normal([1024, n_classes]))}
    
    biases = {'b_conv1': tf.Variable(tf.random_normal([32])),
              'b_conv2': tf.Variable(tf.random_normal([64])),
              'b_fc': tf.Variable(tf.random_normal([1024])),
              'out': tf.Variable(tf.random_normal([n_classes]))}
    
    X = tf.reshape(X, shape=[-1, 28, 28, 1])
    
    conv1 = conv2d(X, weights['w_conv1'])
    conv1 = maxpool2d(conv1)
    
    
    conv2 = conv2d(conv1, weights['w_conv2'])
    conv2 = maxpool2d(conv2)
    
    fc = tf.reshape(conv2, [-1, 7*7*64])
    fc = tf.nn.relu(tf.matmul(fc, weights['w_fc']) + biases['b_fc'])
    
    #fc = tf.nn.dropout(fc, keep_rate)
    
    output = tf.matmul(fc, weights['out']) + biases['out']
    
    return output

#### Function for network training

In [ ]:
def train_neural_network(X):
    prediction = convolutional_neural_networks(X)
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=prediction, labels=y))
    optimizer = tf.train.AdamOptimizer().minimize(cost)
    
    hm_epochs = 10
    with tf.Session() as sess:
        sess.run(tf.initialize_all_variables())
        for epoch in range(hm_epochs):
            epoch_loss = 0
            for _ in range(int(mnist.train.num_examples/batch_size)):
                epoch_x, epoch_y = mnist.train.next_batch(batch_size)
                _, c = sess.run([optimizer, cost], feed_dict = {X:epoch_x, y:epoch_y})
                epoch_loss += c
            print('Epoch: ', epoch, 'completed out of', hm_epochs,'loss: ', epoch_loss)
            
        correct = tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))
        accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
        print('Accuracy', accuracy.eval({X:mnist.test.images, y:mnist.test.labels}))

#### Call training fuction 

In [ ]:
train_neural_network(X)